In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [2]:
wb = load_workbook(filename='Boston Neighborhood Coordinates.xlsx', read_only=True)
ws = wb.active
col_names = [ws["A1"].value, ws["B1"].value, ws["C1"].value, ws["D1"].value]
header = np.array(col_names)

df = pd.DataFrame(ws.values, columns=header)
wb.close()

In [3]:
#df['Zip Code'] = df['Zip Code'].astype(str).str.zfill(5)
df = df.drop(['Zip Code'], axis=1)
df = df.drop([0])
df.set_index('Neighborhood', inplace=True)

In [4]:
df = df.dropna()

In [5]:
df

,Latitude,Longitude
Neighborhood,,
Chinatown/Leather District,42.3515,-71.0602
West End,42.3653,-71.0646
Bay Village,42.3531,-71.0765
South End,42.3378,-71.0706
Roxbury,42.3218,-71.0884
Mission Hill,42.3319,-71.0988
Mattapan,42.2771,-71.0914
South Boston,42.3365,-71.0349
East Boston,42.3695,-71.0201


In [6]:
import folium # map rendering library

In [7]:
# create map of New York using latitude and longitude values
map_Boston = folium.Map(location=[42.3261, -71.07], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df.index):
    label = f'{neighborhood}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Boston)  
    
map_Boston

In [8]:
import requests
import json # library to handle JSON files

In [9]:
CLIENT_ID = 'GKVJ5KSFU1RD00P24CV5JT2XSARMWSONFYRMLT2EK2IJFEBW' # your Foursquare ID
CLIENT_SECRET = 'SSJALJ4S5QXDNQBZMNUNFBE1XMVQZETMJWU1JDXE2VWK2PZL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [10]:
hood_name = 'West End'
hood_lat = df.loc[hood_name, 'Latitude'] # neighborhood latitude value
hood_long = df.loc[hood_name, 'Longitude'] # neighborhood longitude value
radius = 500
LIMIT = 100

url = f'https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={hood_lat},{hood_long}&v={VERSION}&radius={radius}&limit={LIMIT}'

In [11]:
r = requests.get(url)
r.status_code
results = r.json()

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,TD Garden,Hockey Arena,42.366329,-71.062153
1,Bobby Orr Statue @ TD Garden,Outdoor Sculpture,42.365314,-71.062381
2,Tasty Burger,Burger Joint,42.365738,-71.063105
3,The Clubs at Charles River Park,Gym / Fitness Center,42.365985,-71.068257
4,J. Pace & Son,Deli / Bodega,42.364682,-71.067107


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

74 venues were returned by Foursquare.


In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
Boston_venues = getNearbyVenues(names=df.index, latitudes=df['Latitude'], longitudes=df['Longitude'])

Chinatown/Leather District
West End
Bay Village
South End
Roxbury
Mission Hill
Mattapan
South Boston
East Boston
Charlestown
Jamaica Plain
Roslindale
West Roxbury
Allston
Brighton
Hyde Park
North Cambridge 
Kendall Square
Brookline Village
Chestnut Hill
Downtown
North End
Beacon Hill
Back Bay
Dorchester
Harvard Square 
East Cambridge
Fenway/Kenmore
Porter Square
West Cambridge
Inman Square
Central Square
Davis Square
Winter Hill
Assembly Square
Seaport


In [17]:
print(Boston_venues.shape)
    
Boston_venues.nunique()

(1396, 7)


Neighborhood                36
Neighborhood Latitude       36
Neighborhood Longitude      32
Venue                     1169
Venue Latitude            1318
Venue Longitude           1319
Venue Category             248
dtype: int64

In [18]:
Boston_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown/Leather District,42.3515,-71.0602,Gracenote Coffee,42.351657,-71.058204,Coffee Shop
1,Chinatown/Leather District,42.3515,-71.0602,Shabu Zen,42.350922,-71.060942,Asian Restaurant
2,Chinatown/Leather District,42.3515,-71.0602,Crave - Mad for Chicken,42.350378,-71.060786,Korean Restaurant
3,Chinatown/Leather District,42.3515,-71.0602,Shojo,42.351028,-71.060736,Asian Restaurant
4,Chinatown/Leather District,42.3515,-71.0602,Hostelling International Boston,42.351084,-71.063398,Hostel


In [39]:
Venue_cat_count = Boston_venues.groupby('Venue Category').count()
Venue_cat_count.rename(columns={'Venue': 'Count'}, inplace=True)
Venue_cat_count = pd.DataFrame(Venue_cat_count['Count'], Venue_cat_count.index)
Venue_cat_count

,Count
Venue Category,
Accessories Store,3
Afghan Restaurant,2
African Restaurant,1
Airport,1
Airport Lounge,3
...,...
Wine Bar,9
Wine Shop,3
Wings Joint,1


In [40]:
Venue_cat_count.to_excel('Venue_Category_Counts.xlsx')

In [19]:
Venue_per_hood = Boston_venues.groupby('Neighborhood').count()
Venue_per_hood.rename(columns={'Venue': 'Count'}, inplace=True)
Venue_per_hood = pd.DataFrame(Venue_per_hood['Count'], Venue_per_hood.index)
Venue_per_hood

,Count
Neighborhood,
Allston,10
Assembly Square,83
Back Bay,56
Bay Village,50
Beacon Hill,39
Brighton,13
Brookline Village,3
Central Square,38
Charlestown,25


In [20]:
# one hot encoding
Boston_onehot = pd.get_dummies(Boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Boston_onehot['Neighborhood'] = Boston_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Boston_onehot.columns[-1]] + list(Boston_onehot.columns[:-1])
Boston_onehot = Boston_onehot[fixed_columns]

Totals = Boston_onehot.sum()
Totals = pd.DataFrame(Totals)
Totals = Totals.drop('Neighborhood')
Totals.rename(columns={0: 'Totals'}, inplace=True)

In [21]:
Boston_grouped = Boston_onehot.groupby('Neighborhood').mean().reset_index()
Boston_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Allston,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Assembly Square,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.024096,0.000000,...,0.000000,0.000000,0.000000,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048
2,Back Bay,0.017857,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.035714,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bay Village,0.000000,0.020000,0.000000,0.00000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Beacon Hill,0.025641,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.025641,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Brighton,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000
6,Brookline Village,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Central Square,0.026316,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.026316,0.000000,...,0.078947,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Charlestown,0.040000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Chestnut Hill,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
num_top_venues = 10

for hood in Boston_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Boston_grouped[Boston_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allston----
                           venue  freq
0            Rental Car Location   0.2
1                     Donut Shop   0.2
2                            Gym   0.1
3                         Bakery   0.1
4              Convenience Store   0.1
5                    Music Venue   0.1
6                           Park   0.1
7                   Liquor Store   0.1
8                    Pastry Shop   0.0
9  Paper / Office Supplies Store   0.0


----Assembly Square----
                 venue  freq
0       Clothing Store  0.11
1          Coffee Shop  0.04
2  Sporting Goods Shop  0.04
3                  Gym  0.02
4        Shopping Mall  0.02
5           Shoe Store  0.02
6                 Park  0.02
7           Kids Store  0.02
8     Department Store  0.02
9                 Bank  0.02


----Back Bay----
                      venue  freq
0                     Hotel  0.07
1        Seafood Restaurant  0.05
2        Italian Restaurant  0.05
3                     Plaza  0.04
4                Rest

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Boston_grouped['Neighborhood']

for ind in np.arange(Boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Boston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,Donut Shop,Rental Car Location,Liquor Store,Bakery,Gym,Park,Convenience Store,Music Venue,Dessert Shop,Food Court
1,Assembly Square,Clothing Store,Coffee Shop,Sporting Goods Shop,Pizza Place,Department Store,Gym,Kids Store,Shopping Mall,Shoe Store,American Restaurant
2,Back Bay,Hotel,Seafood Restaurant,Italian Restaurant,Ice Cream Shop,Grocery Store,Mediterranean Restaurant,Coffee Shop,Clothing Store,Restaurant,Plaza
3,Bay Village,Spa,Jewelry Store,Park,American Restaurant,Plaza,Coffee Shop,Clothing Store,Salad Place,Hotel,Vegetarian / Vegan Restaurant
4,Beacon Hill,Pizza Place,Italian Restaurant,French Restaurant,Gourmet Shop,Food Truck,Yoga Studio,Breakfast Spot,Burrito Place,Café,Restaurant
5,Brighton,Pizza Place,Supplement Shop,Chinese Restaurant,Bookstore,Convenience Store,Wings Joint,Coffee Shop,Mobile Phone Shop,Park,Sushi Restaurant
6,Brookline Village,Lake,Plaza,Metro Station,Women's Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market
7,Central Square,Vegetarian / Vegan Restaurant,Cocktail Bar,Sandwich Place,Bar,Nightclub,Supermarket,Bookstore,Breakfast Spot,Shanghai Restaurant,Rock Club
8,Charlestown,Coffee Shop,Convenience Store,Pizza Place,Thai Restaurant,Bank,Shopping Mall,Café,Pool,Pharmacy,Pet Store
9,Chestnut Hill,Playground,Park,Women's Store,Donut Shop,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fast Food Restaurant


In [87]:
mask = neighborhoods_venues_sorted.applymap(lambda x: 'Coffee' in str(x))
search_results = neighborhoods_venues_sorted[mask == True].dropna(how='all')

i=0
for items in zip(search_results['Neighborhood']):
    i=i+1

print(f"{i} Results Found!")

17 Results Found!


In [88]:
search_neigh = neighborhoods_venues_sorted.iloc[search_results.index]
search_neigh

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Assembly Square,Clothing Store,Coffee Shop,Sporting Goods Shop,Pizza Place,Department Store,Gym,Kids Store,Shopping Mall,Shoe Store,American Restaurant
2,Back Bay,Hotel,Seafood Restaurant,Italian Restaurant,Ice Cream Shop,Grocery Store,Mediterranean Restaurant,Coffee Shop,Clothing Store,Restaurant,Plaza
3,Bay Village,Spa,Jewelry Store,Park,American Restaurant,Plaza,Coffee Shop,Clothing Store,Salad Place,Hotel,Vegetarian / Vegan Restaurant
5,Brighton,Pizza Place,Supplement Shop,Chinese Restaurant,Bookstore,Convenience Store,Wings Joint,Coffee Shop,Mobile Phone Shop,Park,Sushi Restaurant
8,Charlestown,Coffee Shop,Convenience Store,Pizza Place,Thai Restaurant,Bank,Shopping Mall,Café,Pool,Pharmacy,Pet Store
10,Chinatown/Leather District,Chinese Restaurant,Asian Restaurant,Bakery,Coffee Shop,Sushi Restaurant,Sandwich Place,Hotel Bar,Café,Spa,Dessert Shop
13,Downtown,Hotel,Historic Site,Coffee Shop,Sandwich Place,Park,Steakhouse,Bakery,Greek Restaurant,American Restaurant,Salad Place
14,East Boston,Donut Shop,Coffee Shop,Airport Lounge,American Restaurant,Electronics Store,Airport Service,Café,Hotel,Bus Station,Brewery
15,East Cambridge,Café,Sandwich Place,Hotel,Coffee Shop,Lingerie Store,Food Truck,Gym / Fitness Center,American Restaurant,Mediterranean Restaurant,Park
16,Fenway/Kenmore,Sports Bar,Lounge,Pizza Place,American Restaurant,Coffee Shop,Thai Restaurant,Restaurant,Donut Shop,Greek Restaurant,Japanese Restaurant


In [89]:
mask = search_neigh.applymap(lambda x: 'Sports' in str(x))
search_results_2 = search_neigh[mask == True].dropna(how='all')
search_results_2['Neighborhood'] = neighborhoods_venues_sorted['Neighborhood']

i=0
for items in zip(search_results_2['Neighborhood']):
    i=i+1

print(f"{i} Results Found!")

2 Results Found!


In [90]:
search_neigh_2 = neighborhoods_venues_sorted.iloc[search_results_2.index]
search_neigh_2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Fenway/Kenmore,Sports Bar,Lounge,Pizza Place,American Restaurant,Coffee Shop,Thai Restaurant,Restaurant,Donut Shop,Greek Restaurant,Japanese Restaurant
33,West End,Sandwich Place,Hotel,Coffee Shop,Donut Shop,Bar,Pizza Place,Sports Bar,Brewery,Mexican Restaurant,Café
